---
title: "Lecture 9: Rootfinding methods"
author: "Jamie Haddock"
format: 
    revealjs:
        output-file: Lecture9_slides
        slide-number: true
        chalkboard: 
            buttons: false
        preview-links: auto
        logo: figs/hmc.png
        css: input/slides.css
        incremental: true
        smaller: true
        code-fold: true
    html: 
        code-fold: true
    pdf:
        documentclass: article
        toc: true
        number-sections: true
        geometry:
          - top=1in
          - left=1in
          - bottom=1in
          - right=1in
format-links: false
jupyter: julia-1.9
filters: 
  - input/remove-pause.lua
execute:
  echo: true
  eval: true
---

# Rootfinding problems

*Many* problems in engineering, various sciences, data science, and machine learning can be rephrased as finding a root of a given function!  It is so important an archetypical problem that we study a variety of methods for this generic problem formulation, but you will see that many may be familiar from specific applications where they may go under different names.

## Rootfinding problem

::: {.callout-note icon=false}
## Definition: Rootfinding problem
Given a continuous function $\mathbf{f}$ of a variable input $\mathbf{v}$, the **rootfinding problem** is to find a real input $\mathbf{r}$, called a **root** such that $$\mathbf{f}(\mathbf{r}) = \mathbf{0}.$$
:::

. . .

In Calculus, you have likely already encountered such a problem.  In *optimization*, we often seek a *stationary point* of a given objective function as minima and maxima of 

<!--
[verbose test]{.content-hidden when-format="revealjs" when-format="pptx"}

::: {.callout-caution icon=false}
## Exercise: 

:::

<details><summary>Answer:</summary> </details>


::: {.callout-note icon=false}
## Definition: 
 
:::


::: {.callout-tip icon=false}
## Note: 
 
:::
-->